# Decoding captcha

In [ ]:
from PIL import Image
import os
import matplotlib.pyplot as plt
import numpy as np
from tensorflow import keras
from keras import api
from keras.api import layers
from keras.api.datasets import mnist
from sklearn.model_selection import train_test_split
import cv2 as cv
import sklearn

In [ ]:

file_names = os.listdir('D://kapchi//dataset//dataset')
y_s = [f.split('.')[0] for f in file_names] # Vector of answers to captchas
data = []
for i in range(len(file_names)):
    if len(file_names[i].split('.')[0]) == 5:
        img = (Image.open('D://kapchi//dataset//dataset//' + file_names[i]).convert('L'))
        data.append([np.asarray(img) / 255, y_s[i]])  # We represent the captcha as a normalized, single-channel image, stretched into a vector.
np.random.shuffle(data)
x, y = [], []
for elem in data:
    x.append(elem[0]) # Vector x is a vector whose elements correspond to the vector representation of each image
    y.append(elem[1]) # Vector y is a vector whose elements correspond to the decryption of each captcha.
n_elem = len(x)
print(n_elem)

In [ ]:
x_train = np.expand_dims(np.array(x[:7000]), axis=3)
y_train = np.array(y[:7000])
x_test = np.expand_dims(np.array(x[7000:]), axis=3)
y_test = np.array(y[7000:])
print(len(x_test))

In [ ]:
letters = 'ёйцукенгшщзхъфывапролджэячсмитьбю' + 'ёйцукенгшщзхъфывапролджэячсмитьбю'.upper() + '0123456789'
letters_chars = [ord(l) for l in letters]
y_train_cat = np.array([[api.utils.to_categorical(letters.find(elem[i]), len(letters_chars)) for elem in y_train] for i in range(5)])
y_test_cat = np.array([[api.utils.to_categorical(letters.find(elem[i]), len(letters_chars)) for elem in y_test] for i in range(5)])

# Network architecture

In [ ]:
def create_model():
    img = layers.Input(shape=(60, 200, 1))
    conv1 = layers.Conv2D(16, (3, 3), padding='same', activation='relu')(img) # Convolution1(input(60x200x1), output(60x200x16))
    mp1 = layers.MaxPooling2D(padding='same')(conv1)                          # pooling1(input(60x200x16), output(30x100x16))
    conv2 = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(mp1) # Convolution2(input(30x100x16), output(30x100x32))
    mp2 = layers.MaxPooling2D(padding='same')(conv2)                          # pooling2(input(30x100x32), output(15x50x32))
    conv3 = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(mp2) # Convolution3(input(15x50x32), output(15x50x32))
    bn = layers.BatchNormalization()(conv3)                                   # Batch_norm
    mp3 = layers.MaxPooling2D(padding='same')(bn)                             # pooling3(input(15x50x32), output(8x25x32))
    flat = layers.Flatten()(mp3)                                              # flatten(input(8x25x32), output(6400))
    
    outs = []
    for _ in range(5):                                                        # form five output layers
        dens1 = layers.Dense(64, activation='relu')(flat)                     # Dense[1, 2, 3, 4, 5](input(6400), output(64))
        dr = layers.Dropout(0.6)(dens1)                                       # Dropout(60%)
        res = layers.Dense(76, activation='softmax')(dr)                      # Dense[1, 2, 3, 4, 5](input(64), output(76)-> softmax)
        
        outs.append(res)
    model = api.Model(img, outs)
    model.compile(loss='categorical_crossentropy', 
                  optimizer='adam',
                  metrics=[["accuracy"],["accuracy"],["accuracy"],["accuracy"],["accuracy"]])
    return model


In [ ]:
model = create_model()
model.summary()

 # Training the model

In [ ]:
hist = model.fit(x_train, [y_train_cat[0], y_train_cat[1], y_train_cat[2], y_train_cat[3], y_train_cat[4]], validation_split=0.3, batch_size=50, epochs=60)

# Graphs

Graph of the change in the loss function on training and validation data.
Graph of the change in the "accuracy" metric for the first output of the model.

In [ ]:
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(hist.history['loss'], color='blue')
plt.plot(hist.history['val_loss'], color='black')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend(('loss','val_loss'))
plt.subplot(1, 2, 2)
plt.plot(hist.history['dense_111_accuracy'], color='blue')
plt.plot(hist.history['val_dense_111_accuracy'], color='black')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend(('accuracy','val_accuracy'))
plt.show()

# Testing

Checking the model's performance on a test and train samples.

In [ ]:
pred_train = model.evaluate(x_train, [y_train_cat[0], y_train_cat[1],y_train_cat[2],y_train_cat[3],y_train_cat[4]])
pred_test = model.evaluate(x_test , [y_test_cat[0], y_test_cat[1],y_test_cat[2],y_test_cat[3],y_test_cat[4]])
print(f'loss on training data= {pred_train[0]}')
print(f"loss on testing data= {pred_test[0]}")

We are forming a list of answers (predict)

In [ ]:
def get_result(x, i):
    return ''.join([letters[np.argmax(x[j][i])] for j in range(5)])
predict = np.array(model.predict(x_test))
predict = np.array([get_result(predict, i) for i in range(predict.shape[1])])

Number of correct and incorrect classifications

In [ ]:
mask = y_test == predict
x_correct = x_test[mask]
y_correct = predict[mask]
x_incorrect = x_test[~mask]
y_incorrect = predict[~mask]
answ = y_test[~mask]
print(f"number of correct classifications = {len(x_correct)}")
print(f"number of incorrect classifications = {len(x_incorrect)}")


# Images of 4 correct classifications

In [ ]:
plt.figure(figsize=(10, 5))
for i in range(1, 5):
    plt.subplot(2, 2, i)
    plt.imshow(x_correct[i-1], cmap=plt.cm.binary)
    plt.title(f"predict={y_correct[i-1]}")


# Images of 4 incorrect classifications

In [ ]:
plt.figure(figsize=(10, 5))
for i in range(1, 5):
    plt.subplot(2, 2, i)
    plt.imshow(x_incorrect[i-1], cmap=plt.cm.binary)
    plt.title(f"predict={y_incorrect[i-1]}, answer={answ[i-1]}")


# Save model

In [ ]:
model_name = 'CaptchaDecoder.h5'
MODELS_DIR = 'D:\work\project2\pythonProject'
# Save model and weights
if not os.path.isdir(MODELS_DIR):
    os.makedirs(MODELS_DIR)
model_path = os.path.join(MODELS_DIR, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)